Generalnie kluczowa zmiana od miejsca, gdzie jest napisane !!!!Przełom
po drodze jakieś eksperymenty z ustalaniem typow
Nowa funkcja cleaningRDD3 zmieniająca na DF i jakos sie nie wykrzacza - nie wiem, czy jest wydajna, ale cos robi

Kolejny etap to co sie da pozamieniac na inty, połaczyć pare tabelek i ułożyć do wszystkiego historię

In [122]:
import findspark
findspark.init()

In [123]:
from pyspark import SparkContext, SparkConf

In [124]:
conf=SparkConf().setAppName("BDProject").setMaster("local[3]")

In [125]:
sc=SparkContext(conf=conf)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=BDProject, master=local[3]) created by __init__ at <ipython-input-4-38871edff11b>:1 

In [126]:
rawBadges=sc.textFile("travel.stackexchange.com\Badges.xml")

In [127]:
rawUsers=sc.textFile("travel.stackexchange.com\\Users.xml")

In [128]:
rawUsers.take(3)

['<?xml version="1.0" encoding="utf-8"?>',
 '<users>',
 '  <row Id="-1" Reputation="1" CreationDate="2011-06-21T15:16:44.000" DisplayName="Community" LastAccessDate="2011-06-21T15:16:52.063" Location="on the server farm" AboutMe="&lt;p&gt;Hi, I\'m not really a person.&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I\'m a background process that helps keep this site clean!&lt;/p&gt;&#xA;&#xA;&lt;p&gt;I do things like&lt;/p&gt;&#xA;&#xA;&lt;ul&gt;&#xA;&lt;li&gt;Randomly poke old unanswered questions every hour so they get some attention&lt;/li&gt;&#xA;&lt;li&gt;Own community questions and answers so nobody gets unnecessary reputation from them&lt;/li&gt;&#xA;&lt;li&gt;Own downvotes on spam/evil posts that get permanently deleted&lt;/li&gt;&#xA;&lt;li&gt;Own suggested edits from anonymous users&lt;/li&gt;&#xA;&lt;li&gt;&lt;a href=&quot;http://meta.stackexchange.com/a/92006&quot;&gt;Remove abandoned questions&lt;/a&gt;&lt;/li&gt;&#xA;&lt;/ul&gt;&#xA;" Views="30" UpVotes="21" DownVotes="17" Age="7" AccountId=

In [93]:
def cleaningRDD(RDDIn):
    def cleaning(listIn):
        listOut=[]
        for x in listIn:
            if listIn.index(x)%2==0:
                listOut.append(x.replace(' ','').replace('<','').replace('=',''))
            else:
                listOut.append(x)
        return {listOut[i]:listOut[i+1] for i in range(0,len(listOut)-1,2)}
    return RDDIn.filter(lambda x: x.startswith('  <row Id=')).\
map(lambda x: x.split('"')).map(cleaning)

In [9]:
badges=cleaningRDD(rawBadges)

In [10]:
badges.take(2)

[{'Class': '3',
  'Date': '2011-06-21T20:16:50.113',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '2',
  'rowId': '1'},
 {'Class': '3',
  'Date': '2011-06-21T20:16:50.113',
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': '4',
  'rowId': '2'}]

In [24]:
def is_number_tryexcept(s):
    """ Returns True is string is a number. """
    try:
        int(s)
        return True
    except ValueError:
        return False

In [20]:
slownik={'Class': '-3','anyDate': '2011-06-21T20:16:48.910','Name': 'Autobiographer','TagBased': 'False','UserId': '2-','rowId': '1'}

In [83]:
slownik2={'Class': '-3','anyDate': '2011-06-21T20:16:48.910','UserId': '2-','rowId': '1','Name': 'Autobiographer'}

In [32]:
for x in slownik.keys():
    if is_number_tryexcept(slownik[x])==True:
        slownik[x]=int(slownik[x])

In [22]:
slownik.keys()

dict_keys(['Class', 'anyDate', 'Name', 'TagBased', 'UserId', 'rowId'])

In [126]:
def digitowanie(slownik):
    from datetime import datetime
    for x in slownik.keys():
        if slownik[x].replace('-','').isdigit():
            slownik[x]=int(slownik[x])
        elif x.find('Date')>0:
            slownik[x]=datetime.strptime(slownik[x][0:-4],'%Y-%m-%dT%H:%M:%S')
    return slownik

In [88]:
rowSchema=Row(**slownik)

In [89]:
rowSchema

Row(Class='-3', Name='Autobiographer', TagBased='False', UserId='2-', anyDate='2011-06-21T20:16:48.910', rowId='1')

In [90]:
rowSchema(slownik)

Row(-3={'Class': '-3', 'anyDate': '2011-06-21T20:16:48.910', 'Name': 'Autobiographer', 'TagBased': 'False', 'UserId': '2-', 'rowId': '1'})

In [42]:
x=rowSchema(*slownik.values())

In [43]:
x

Row(Class='-3', anyDate='2011-06-21T20:16:48.910', Name='Autobiographer', TagBased='False', UserId='2-', rowId='1')

In [116]:
x='2017-01-02dsdsds'
y=datetime.strptime(x,'%Y-%d-%mdsdsds')
y

datetime.datetime(2017, 2, 1, 0, 0)

In [132]:
slownik.values()

dict_values([3, datetime.datetime(2011, 6, 21, 20, 16, 48), 'Autobiographer', 'False', 2, 1])

In [131]:
digitowanie(slownik)

{'Class': 3,
 'Name': 'Autobiographer',
 'TagBased': 'False',
 'UserId': 2,
 'anyDate': datetime.datetime(2011, 6, 21, 20, 16, 48),
 'rowId': 1}

In [133]:
slownik['anyDate']

datetime.datetime(2011, 6, 21, 20, 16, 48)

In [7]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [8]:
spark = SparkSession.builder \
    .appName('DataFrame_1') \
    .master('local[*]') \
    .getOrCreate()

In [98]:
badgesInt=badges.map(lambda x: digitowanie(x))

In [99]:
type(badgesInt)

pyspark.rdd.PipelinedRDD

In [101]:
badgesDF=badgesInt.map(lambda x: Row(**x)).toDF()

In [102]:
badgesDF.printSchema()

root
 |-- Class: long (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Name: string (nullable = true)
 |-- TagBased: string (nullable = true)
 |-- UserId: long (nullable = true)
 |-- rowId: long (nullable = true)



In [106]:
badges2=cleaningRDD2(rawBadges)
badges2.take(2)

[{'Class': 3,
  'Date': datetime.datetime(2011, 6, 21, 20, 16, 50),
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': 2,
  'rowId': 1},
 {'Class': 3,
  'Date': datetime.datetime(2011, 6, 21, 20, 16, 50),
  'Name': 'Autobiographer',
  'TagBased': 'False',
  'UserId': 4,
  'rowId': 2}]

In [9]:
def cleaningRDD2(RDDIn):
    def cleaning(listIn):
        listOut=[]
        for x in listIn:
            if listIn.index(x)%2==0:
                listOut.append(x.replace(' ','').replace('<','').replace('=',''))
            else:
                listOut.append(x)
        return {listOut[i]:listOut[i+1] for i in range(0,len(listOut)-1,2)}
    def isItNumber(s):
        """ Returns True if string is a number. """
        try:
            int(s)
            return True
        except ValueError:
            return False
    def digitowanie(slownik):
        from datetime import datetime
        for x in slownik.keys():
            if isItNumber(slownik[x])==True:
                slownik[x]=int(slownik[x])
            elif x.find('Date')>0:
                slownik[x]=datetime.strptime(slownik[x][0:-4],'%Y-%m-%dT%H:%M:%S')
        return slownik
    return RDDIn.filter(lambda x: x.startswith('  <row Id=')).\
map(lambda x: x.split('"')).map(cleaning).map(digitowanie)

In [33]:
from pyspark.sql.types import IntegerType, StringType, StructType, StructField,\
TimestampType, BooleanType

In [50]:
df.printSchema()

root
 |-- Class: integer (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Name: string (nullable = true)
 |-- TagBased: string (nullable = true)
 |-- UserId: integer (nullable = true)
 |-- rowId: integer (nullable = true)



In [52]:
users2=cleaningRDD2(rawUsers)

In [53]:
users2DF=users2.map(lambda x: Row(**x)).toDF()

In [55]:
users2DF.printSchema()

root
 |-- AboutMe: string (nullable = true)
 |-- AccountId: long (nullable = true)
 |-- Age: long (nullable = true)
 |-- CreationDate: timestamp (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- DownVotes: long (nullable = true)
 |-- LastAccessDate: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Reputation: long (nullable = true)
 |-- UpVotes: long (nullable = true)
 |-- Views: long (nullable = true)
 |-- rowId: long (nullable = true)



In [15]:
users3=users2.flatMap(lambda x: list(x.keys())).distinct().collect()

!!!!!Przełom:-)

Tutaj mamy nową wersje funkcji - z danego RDD robi Data Frame ze stringami na razie, ale radzi sobie ze słownikami z niekompletnymi polami

In [118]:
def cleaningRDD3(RDDIn):
    def preCleanUp(RDDIn):
        def cleaning(listIn):
            listOut=[]
            for x in listIn:
                if listIn.index(x)%2==0:
                    listOut.append(x.replace(' ','').replace('<','').replace('=',''))
                else:
                    listOut.append(x)
            return {listOut[i]:listOut[i+1] for i in range(0,len(listOut)-1,2)}
        return RDDIn.filter(lambda x: x.startswith('  <row Id=')).map(lambda x: x.split('"')).map(cleaning)
    dictRDD=preCleanUp(RDDIn)
    listOfColums=dictRDD.flatMap(lambda x: list(x.keys())).distinct().collect()
    def newDict(dictIn):
        for col in listOfColums:
            if col not in dictIn.keys():
                dictIn[col]=None
        return dictIn
    #nie mam bladego pojęcia dlaczego dziadostwo nie działa bez sampleRatio
    return dictRDD.map(newDict).map(lambda x: Row(**x)).toDF(sampleRatio=100)

In [68]:
#to ma byc funkcja ktora dodaje brakujace pola do slownika, zeby pozniej rowy mialy tyle samo wartosci
def newDictTest(dictIn,listOfColums):
    for col in listOfColums:
        if col not in dictIn.keys():
            dictIn[col]=None
    return dictIn

In [69]:
testDict={'a':1,'b':2,'d':22}
testCol=['a','ba','s','w','da']

In [70]:
newDictTest(testDict,testCol)

{'a': 1, 'b': 2, 'ba': None, 'd': 22, 'da': None, 's': None, 'w': None}

In [119]:
users4=cleaningRDD3(rawUsers)

In [120]:
type(users4)

pyspark.sql.dataframe.DataFrame

In [121]:
users4.printSchema()

root
 |-- AboutMe: string (nullable = true)
 |-- AccountId: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- DownVotes: string (nullable = true)
 |-- LastAccessDate: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- ProfileImageUrl: string (nullable = true)
 |-- Reputation: string (nullable = true)
 |-- UpVotes: string (nullable = true)
 |-- Views: string (nullable = true)
 |-- WebsiteUrl: string (nullable = true)
 |-- rowId: string (nullable = true)



In [95]:
users4.show(1)

+--------------------+---------+---+--------------------+-----------+---------+--------------------+------------------+---------------+----------+-------+-----+----------+-----+
|             AboutMe|AccountId|Age|        CreationDate|DisplayName|DownVotes|      LastAccessDate|          Location|ProfileImageUrl|Reputation|UpVotes|Views|WebsiteUrl|rowId|
+--------------------+---------+---+--------------------+-----------+---------+--------------------+------------------+---------------+----------+-------+-----+----------+-----+
|&lt;p&gt;Hi, I'm ...|       -1|  7|2011-06-21T15:16:...|  Community|       17|2011-06-21T15:16:...|on the server farm|           null|         1|     21|   30|      null|   -1|
+--------------------+---------+---+--------------------+-----------+---------+--------------------+------------------+---------------+----------+-------+-----+----------+-----+
only showing top 1 row



In [96]:
users4.columns

['AboutMe',
 'AccountId',
 'Age',
 'CreationDate',
 'DisplayName',
 'DownVotes',
 'LastAccessDate',
 'Location',
 'ProfileImageUrl',
 'Reputation',
 'UpVotes',
 'Views',
 'WebsiteUrl',
 'rowId']

In [99]:
users4.select('DisplayName','Reputation', 'Age').where(users4.Age.isNull()).show()

+-------------------+----------+----+
|        DisplayName|Reputation| Age|
+-------------------+----------+----+
|     Kevin Montrose|       101|null|
|          user27478|      4550|null|
|          mrigasira|       904|null|
|             jeroen|       306|null|
|          Gagravarr|     45427|null|
|          HedgeMage|       370|null|
|            Rodolfo|      1202|null|
|    theycallmemorty|       441|null|
|                Ben|       178|null|
|       Joel Spolsky|      7203|null|
|Francesca Venturini|       140|null|
|                Eri|       310|null|
|            Workman|       101|null|
|            Vitalik|       184|null|
|              fmark|       101|null|
|               z  -|       260|null|
|                nic|      5045|null|
|             Beaker|      3608|null|
|   TheEnigmaMachine|       658|null|
|            Sufendy|      2439|null|
+-------------------+----------+----+
only showing top 20 rows



In [100]:
users4.select('DisplayName','Age','Reputation').show()

+----------------+----+----------+
|     DisplayName| Age|Reputation|
+----------------+----+----------+
|       Community|   7|         1|
|    Geoff Dalgas|  41|       101|
|     Nick Craver|  33|       101|
|          Emmett|  32|       101|
|  Kevin Montrose|null|       101|
|   Jimmy Sawczuk|  31|       101|
|        Seanland|  33|      1206|
|    silent1mezzo|  31|      1978|
|     Darren Kopp|  33|      1182|
|       user27478|null|      4550|
|      daybreaker|  37|       466|
|    Matthew Read|  31|      1763|
|         cerealk|  40|       101|
|       mrigasira|null|       904|
|           VMAtm|  34|     19165|
|Rebecca Chernoff|  34|       101|
|          jeroen|null|       306|
|   Michael Pryor|  42|      1684|
|       Gagravarr|null|     45427|
|          Nicole|  36|       707|
+----------------+----+----------+
only showing top 20 rows



Test na Tagach

In [101]:
rawTags=sc.textFile("travel.stackexchange.com\Tags.xml")

In [102]:
tagsDF=cleaningRDD3(rawTags)

In [103]:
tagsDF.printSchema()

root
 |-- Count: string (nullable = true)
 |-- ExcerptPostId: string (nullable = true)
 |-- TagName: string (nullable = true)
 |-- WikiPostId: string (nullable = true)
 |-- rowId: string (nullable = true)



In [106]:
tagsDF.select('Count','TagName','WikiPostId').where(tagsDF.TagName=='argentina').show()

+-----+---------+----------+
|Count|  TagName|WikiPostId|
+-----+---------+----------+
|   92|argentina|      2393|
+-----+---------+----------+



Test na Comments - początkowo się nie udał :-( ale po ustaleniu sampleRatio=100 znowu sie udalo, tylko nie wiadomo dlaczego

In [107]:
rawComments=sc.textFile("travel.stackexchange.com\Comments.xml")

In [109]:
rawComments.take(3)

['<?xml version="1.0" encoding="utf-8"?>',
 '<comments>',
 '  <row Id="1" PostId="8" Score="1" Text="Indeed, recommendations in general are off-topic.  [This page](http://travel.stackexchange.com/questions/how-to-ask-beta) lists a couple blog posts that explain exactly why these questions aren\'t a good fit for the network." CreationDate="2011-06-21T23:26:02.317" UserId="20" />']

In [115]:
comments=cleaningRDD3(rawComments)

In [113]:
rawComments.toDF?

In [117]:
comments.printSchema()

root
 |-- CreationDate: string (nullable = true)
 |-- PostId: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Text: string (nullable = true)
 |-- UserDisplayName: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- rowId: string (nullable = true)

